# M2177.003100 Deep Learning <br> Assignment #4 Implementing Variational AutoEncoder - part1 MNIST data

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Chaehun Shin, September 2020

In this notebook, you will learn how to implement Variational AutoEncoder (VAEs) <br>
The goal here is to build VAEs that draw a digit (MNIST data) <br> 

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all parts**, run the *CollectSubmission.sh* script with your **Student_ID** as input argument. <br>
This will produce a zipped file called *[Your Student_ID].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* &nbsp; Student_ID)

### Some helpful tutorials and references for assignment #4-1:
- [1] Pytorch official tutorials. [[link]](https://pytorch.org/tutorials/)
- [2] Stanford CS231n lectures. [[link]](http://cs231n.stanford.edu/)
- [3] Kingma, Diederik P., and Max Welling. "Auto-encoding variational bayes." arXiv preprint arXiv:1312.6114 (2013).
- [4] Doersch, Carl. "Tutorial on variational autoencoders." arXiv preprint arXiv:1606.05908 (2016).
- [5] Kingma, Diederik P., and Max Welling. "An Introduction to Variational Autoencoders." arXiv preprint arXiv:1906.02691 (2019).

## 0. Download and load MNIST datasets
The MNIST datasets will be downloaded into the 'data/mnist' directory. If you want to change the directory the data is saved in, change 'mnist_data_dir' with where you want. <br>

In [ ]:
import os
from torchvision.datasets import MNIST
import torchvision.transforms as T

mnist_data_dir = './data/mnist'
dataset = MNIST(root=mnist_data_dir,
               transform=T.ToTensor(), train=True, download=True)
print(dataset.train_data.shape)
print(dataset.train_labels.shape)

## <a name="1"></a> 1. Building a network

In this section, you will implement neural networks for <br>
(1) encoder $q_\phi(z \mid x)$ to encode latent variable distribution from the image of the digit <br>
(2) decoder $p_\theta(x \mid z)$ to decode the image distribution of the digit from the sample of latent variable distribution.<br>
You can use some layer function implemented in **'torch.nn'** library (abbretivated as **nn**) or **'torch.nn.functional'** library (abbreviated as **F**) as you want.

Just write the code in whatever way you find most clear.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

It is time for an encoder. <br>
It takes img_data, img_dim(should be 28\*28\*1=784 here), latent_dim(dimension of the z).<br>
It should return parameters of the latent variable distribution with the dimension of latent_dim. <br>
Because we model the latent variable distribution as multivariate Gaussian, we can make the distribution with only mean and covariance matrix. Also as we model each dimension of the latent variable as independent, covariance matrix becomes a diagonal matrix and we need only latent_dim number of elements for covariance matrix. <br>
So we output 2 latent_dim dimension vectors from the encoder function implemented by neural network. **I recommend to use logvar not variance itself for stable training.**


Maybe you can use two neural networks for mean and logvar vectors. However **it is recommended to use only one neural network with two last branches.**<br>
For example, if you use 5 layer networks, first 4 layers are shared and there are two 5th layers each corresponding to mean and logvar.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, img_dim=784, latent_dim=30):
        super().__init__()
        ################ ToDo ################
        
    def forward(self, imgs):
        ################ ToDo ################
        
        return mu, logvar

Now, it's time for a decoder model. <br>
It takes sample of the latent variable distribution(z), latent_dim(dimension of the latent variable), and img_dim(28\*28\*1=784 here). It should return the reconstruction of the original image x_hat. <br>
Decoder models the likelihood distributions and in here, we model each pixel as Bernoulli distribution(So we use binary cross entropy as loss function). So **you should use the sigmoid function to make the output as probability in [0, 1]**

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=30, img_dim=784):
        super().__init__()
        ################ ToDo ################
        
    def forward(self, z):
        ################ ToDo ################
        
        return out

Now, we construct the VAEs with encoder and decoder.<br>
This model rquires an implementation of reparameterization trick (which you need to implement) due to impossible differentiation of sampling.<br>

### Reparameterization trick can be implemented as follows.
Our Encoder outputs mean and variance of Guassian distrubiton as parameters of posterior distribution.<br>

1. We sample epsilon form standard normal distribution with the size of latent dimension. <br>
2. Then, we scale and shift epsilon by using variance and mean of posterior distrubiton.<br>

Because sampling is out of graph, gradient can be flowed from loss to encoder and decoder. So we can train all networks jointly as you see in the forward function.

If you want to sample from the VAEs, you just sample from the prior distribution and go trough the decoder, as you can see in the sample function implemented below.

In [ ]:
class VAE(nn.Module):
    def __init__(self, img_dim=784, latent_dim=10):
        super().__init__()
        self.encoder = Encoder(img_dim, latent_dim)
        self.decoder = Decoder(latent_dim, img_dim)
        self.latent_dim = latent_dim
        
    def encode(self, imgs):
        mu, logvar = self.encoder(imgs)
        return mu, logvar
    
    def decode(self, z):
        recon_imgs = self.decoder(z)
        return recon_imgs
        
    def reparameterize(self, eps, mu, logvar):
        ################ ToDo ################
        
    def forward(self, imgs):
        mu, logvar = self.encode(imgs)
        eps = torch.randn_like(mu)
        z = self.reparameterize(eps, mu, logvar)
        recon_imgs = self.decode(z)
        return recon_imgs, mu, logvar
    
    def sample(self, num_samples=16):
        device = next(self.parameters()).device
        eps = torch.randn((num_samples, self.latent_dim)).to(device)
        gen_imgs = self.decode(eps)
        return gen_imgs        

## <a name="2"></a> 2. Build a main part and train

In this section, you will implement the main part (define the loss function in TODO part).
Feel free to set the hyperparmeters and fill in the main part.
Then run the code and check the model reconstructs a digit.
When you are done, run the following to check your implementations.

Be sure to define **batch size bigger than 16** (Because, we visualize 16 images per batch in training time). <br>
You must show **at least three generated images** (At the beginning of ,in the midway of, at the end of training).

In [ ]:
# hyperparameter setting
img_dim=784
latent_dim=50

batch_size = 64 # it should be larger than 16
learning_rate = 1e-4
total_iter = 50000

log_freq = 10
viz_freq = 50

In [ ]:
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
dataloader_iter = iter(dataloader)

vae = VAE(img_dim, latent_dim).to(device)

optimizer = torch.optim.Adam(vae.parameters(), learning_rate)


for it in range(total_iter):
    try:
        imgs, _ = next(dataloader_iter)
    except:
        dataloader_iter = iter(dataloader)
        imgs, _ = next(dataloader_iter)
        
    imgs = imgs.view((-1, img_dim)).to(device)
    recon_imgs, mu, logvar = vae(imgs)
    
    ################ ToDo ################
    recon_loss = 
    kldiv_loss = 
    total_loss = (recon_loss + kldiv_loss)
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if (it+1) % log_freq == 0:
        print("Iter: %05d/%d, Recon loss: %.4f, KL div loss: %.4f, Total loss: %.4f"%(it+1, total_iter,
                                                                                            recon_loss.data.item(),
                                                                                            kldiv_loss.data.item(),
                                                                                            total_loss.data.item()))
        
    if (it+1) % viz_freq == 0:
        with torch.no_grad():
            gen_imgs = vae.sample(16)
        org_imgs = make_grid(imgs[:16, :].view((16, 1, 28, 28)), nrow=4).permute(1, 2, 0).cpu().detach().numpy()
        plt.subplot(1, 3, 1)
        plt.imshow(org_imgs)
        recon_imgs = make_grid(recon_imgs[:16].view((16, 1, 28, 28)), nrow=4).permute(1, 2, 0).cpu().detach().numpy()
        plt.subplot(1, 3, 2)
        plt.imshow(recon_imgs)
        gen_imgs = make_grid(gen_imgs[:16].view((16, 1, 28, 28)), nrow=4).permute(1, 2, 0).cpu().detach().numpy()
        plt.subplot(1, 3, 3)
        plt.imshow(gen_imgs)
            
        plt.show()